In [42]:
import pandas as pd

In [44]:
df = pd.read_csv('Ailments/Dataset_Aliments/13100196.csv')

In [45]:
# 데이터 확인
df.head()

,REF_DATE,GEO,DGUID,Sex,Age group,Ailments,Frequency,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1990,Austria,NaN,Males,11 years,Headache,Often,Percent,239,units,0,v664448,1.1.1.1.1,6,NaN,NaN,t,0
1,1990,Austria,NaN,Males,11 years,Headache,Sometimes,Percent,239,units,0,v664449,1.1.1.1.4,8,NaN,NaN,t,0
2,1990,Austria,NaN,Males,11 years,Headache,Seldom,Percent,239,units,0,v664450,1.1.1.1.6,30,NaN,NaN,t,0
3,1990,Austria,NaN,Males,11 years,Headache,Never,Percent,239,units,0,v664451,1.1.1.1.7,55,NaN,NaN,t,0
4,1990,Austria,NaN,Males,11 years,Stomach ache,Often,Percent,239,units,0,v665000,1.1.1.2.1,4,NaN,NaN,t,0


# 데이터 전처리
## 필요 요소
1. 사용하지 않는 값 제거
2. NaN값 제거
3. 남성, 여성 구분
4. 나이 값 숫자로 변환
5. Frequency 정규화
6. Ailments 표준화
7. Terminated Boolean 변환
8. Value값을 각각의 행으로 복제


In [46]:
# 사망여부 one hot encoding
df['TERMINATED'] = df['TERMINATED'].apply(lambda x: 'True' if x == 't' else 'False')

In [47]:
df.head()

,REF_DATE,GEO,DGUID,Sex,Age group,Ailments,Frequency,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1990,Austria,NaN,Males,11 years,Headache,Often,Percent,239,units,0,v664448,1.1.1.1.1,6,NaN,NaN,True,0
1,1990,Austria,NaN,Males,11 years,Headache,Sometimes,Percent,239,units,0,v664449,1.1.1.1.4,8,NaN,NaN,True,0
2,1990,Austria,NaN,Males,11 years,Headache,Seldom,Percent,239,units,0,v664450,1.1.1.1.6,30,NaN,NaN,True,0
3,1990,Austria,NaN,Males,11 years,Headache,Never,Percent,239,units,0,v664451,1.1.1.1.7,55,NaN,NaN,True,0
4,1990,Austria,NaN,Males,11 years,Stomach ache,Often,Percent,239,units,0,v665000,1.1.1.2.1,4,NaN,NaN,True,0


In [48]:
# NaN값 제거
df = df.dropna(axis=1)

In [49]:
df.head()

,REF_DATE,GEO,Sex,Age group,Ailments,Frequency,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,TERMINATED,DECIMALS
0,1990,Austria,Males,11 years,Headache,Often,Percent,239,units,0,v664448,1.1.1.1.1,6,True,0
1,1990,Austria,Males,11 years,Headache,Sometimes,Percent,239,units,0,v664449,1.1.1.1.4,8,True,0
2,1990,Austria,Males,11 years,Headache,Seldom,Percent,239,units,0,v664450,1.1.1.1.6,30,True,0
3,1990,Austria,Males,11 years,Headache,Never,Percent,239,units,0,v664451,1.1.1.1.7,55,True,0
4,1990,Austria,Males,11 years,Stomach ache,Often,Percent,239,units,0,v665000,1.1.1.2.1,4,True,0


In [50]:
# decimals, UOM, UOM_ID, SCALA_FACTOR, SCALAR_ID, VECTOR, COORDINATE, 제거
df_preprocess = df.drop(['DECIMALS', 'UOM', 'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE'], axis=1)

In [51]:
df_preprocess.head()

,REF_DATE,GEO,Sex,Age group,Ailments,Frequency,VALUE,TERMINATED
0,1990,Austria,Males,11 years,Headache,Often,6,True
1,1990,Austria,Males,11 years,Headache,Sometimes,8,True
2,1990,Austria,Males,11 years,Headache,Seldom,30,True
3,1990,Austria,Males,11 years,Headache,Never,55,True
4,1990,Austria,Males,11 years,Stomach ache,Often,4,True


In [52]:
# 남성이면 1, 여성이면 0
df_preprocess['Sex'] = df_preprocess['Sex'].apply(lambda  x: 1 if x == 'Males' else 0)

In [53]:
df_preprocess.head()

,REF_DATE,GEO,Sex,Age group,Ailments,Frequency,VALUE,TERMINATED
0,1990,Austria,1,11 years,Headache,Often,6,True
1,1990,Austria,1,11 years,Headache,Sometimes,8,True
2,1990,Austria,1,11 years,Headache,Seldom,30,True
3,1990,Austria,1,11 years,Headache,Never,55,True
4,1990,Austria,1,11 years,Stomach ache,Often,4,True


In [54]:
# 'VALUE' 열의 값만큼 데이터를 복제하는 함수를
def duplicate_rows(df, row_count_column):
    return df.loc[df.index.repeat(df[row_count_column])].drop(columns=row_count_column).reset_index(drop=True)

# 'VALUE' 열을 사용하여 데이터를 복제
expanded_df = duplicate_rows(df_preprocess, 'VALUE')

In [55]:
expanded_df.head()

,REF_DATE,GEO,Sex,Age group,Ailments,Frequency,TERMINATED
0,1990,Austria,1,11 years,Headache,Often,True
1,1990,Austria,1,11 years,Headache,Often,True
2,1990,Austria,1,11 years,Headache,Often,True
3,1990,Austria,1,11 years,Headache,Often,True
4,1990,Austria,1,11 years,Headache,Often,True


In [56]:
# ~~ years 제거하고 숫자만 남기기
expanded_df['Age group'] = expanded_df['Age group'].apply(lambda x: x.split(' ')[0])

In [57]:
expanded_df.head()

,REF_DATE,GEO,Sex,Age group,Ailments,Frequency,TERMINATED
0,1990,Austria,1,11,Headache,Often,True
1,1990,Austria,1,11,Headache,Often,True
2,1990,Austria,1,11,Headache,Often,True
3,1990,Austria,1,11,Headache,Often,True
4,1990,Austria,1,11,Headache,Often,True


In [58]:
# Ailments 데이터 종류 확인    
expanded_df['Ailments'].unique()

array(['Headache', 'Stomach ache', 'Backache', 'Feeling low (depressed)',
       'Bad temper (irritability)', 'Feeling nervous',
       'Difficulty getting to sleep', 'Feeling dizzy',
       'Feeling tired in the morning'], dtype=object)

In [59]:
# 증상 표준화
expanded_df['Ailments'] = expanded_df['Ailments'].apply(lambda x: 'Depressed' if x == 'Feeling low (depressed)' else x)
expanded_df['Ailments'] = expanded_df['Ailments'].apply(lambda x: 'Irritability' if x == 'Bad temper (irritability)' else x)
expanded_df['Ailments'] = expanded_df['Ailments'].apply(lambda x: 'Insomnia' if x == 'Difficulty getting to sleep' else x)
expanded_df['Ailments'] = expanded_df['Ailments'].apply(lambda x: 'Stomachache' if x == 'Stomach ache' else x)
expanded_df['Ailments'] = expanded_df['Ailments'].apply(lambda x: 'Anxiety' if x == 'Feeling nervous' else x)
expanded_df['Ailments'] = expanded_df['Ailments'].apply(lambda x: 'Tiredness' if x == 'Feeling tired in the morning' else x)
expanded_df['Ailments'] = expanded_df['Ailments'].apply(lambda x: 'Dizziness' if x == 'Feeling dizzy' else x)

In [60]:
expanded_df['Ailments'].unique()

array(['Headache', 'Stomachache', 'Backache', 'Depressed', 'Irritability',
       'Anxiety', 'Insomnia', 'Dizziness', 'Tiredness'], dtype=object)

In [61]:
# ailment 데이터를 one-hot encoding
expanded_df = pd.get_dummies(expanded_df, columns=['Ailments'])

In [62]:
expanded_df.head()

,REF_DATE,GEO,Sex,Age group,Frequency,TERMINATED,Ailments_Anxiety,Ailments_Backache,Ailments_Depressed,Ailments_Dizziness,Ailments_Headache,Ailments_Insomnia,Ailments_Irritability,Ailments_Stomachache,Ailments_Tiredness
0,1990,Austria,1,11,Often,True,False,False,False,False,True,False,False,False,False
1,1990,Austria,1,11,Often,True,False,False,False,False,True,False,False,False,False
2,1990,Austria,1,11,Often,True,False,False,False,False,True,False,False,False,False
3,1990,Austria,1,11,Often,True,False,False,False,False,True,False,False,False,False
4,1990,Austria,1,11,Often,True,False,False,False,False,True,False,False,False,False


In [63]:
# frequencty data 종류 확인
expanded_df['Frequency'].unique()

array(['Often', 'Sometimes', 'Seldom', 'Never', '4 or more times a week',
       '1 to 3 times a week', 'Once in a while', 'Seldom/never'],
      dtype=object)

In [64]:
# Never - 0, Seldom - 1, Seldom/never - 1, Once in a while - 2, Sometimes - 3, Often - 4, 1 to 3 times a week - 2, 4 or more times a week - 5로 변환
expanded_df['Frequency'] = expanded_df['Frequency'].apply(lambda x: 0 if x == 'Never' else x)
expanded_df['Frequency'] = expanded_df['Frequency'].apply(lambda x: 1 if x == 'Seldom' else x)
expanded_df['Frequency'] = expanded_df['Frequency'].apply(lambda x: 1 if x == 'Seldom/never' else x)
expanded_df['Frequency'] = expanded_df['Frequency'].apply(lambda x: 2 if x == 'Once in a while' else x)
expanded_df['Frequency'] = expanded_df['Frequency'].apply(lambda x: 3 if x == 'Sometimes' else x)
expanded_df['Frequency'] = expanded_df['Frequency'].apply(lambda x: 4 if x == 'Often' else x)
expanded_df['Frequency'] = expanded_df['Frequency'].apply(lambda x: 2 if x == '1 to 3 times a week' else x)
expanded_df['Frequency'] = expanded_df['Frequency'].apply(lambda x: 5 if x == '4 or more times a week' else x)

In [65]:
expanded_df['Frequency'].unique()

array([4, 3, 1, 0, 5, 2])

In [66]:
expanded_df.head()

,REF_DATE,GEO,Sex,Age group,Frequency,TERMINATED,Ailments_Anxiety,Ailments_Backache,Ailments_Depressed,Ailments_Dizziness,Ailments_Headache,Ailments_Insomnia,Ailments_Irritability,Ailments_Stomachache,Ailments_Tiredness
0,1990,Austria,1,11,4,True,False,False,False,False,True,False,False,False,False
1,1990,Austria,1,11,4,True,False,False,False,False,True,False,False,False,False
2,1990,Austria,1,11,4,True,False,False,False,False,True,False,False,False,False
3,1990,Austria,1,11,4,True,False,False,False,False,True,False,False,False,False
4,1990,Austria,1,11,4,True,False,False,False,False,True,False,False,False,False


In [67]:
# Frequency 데이터 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
expanded_df['Frequency'] = scaler.fit_transform(expanded_df[['Frequency']])

/Users/taemmini/anaconda3/envs/Depression Analysis/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/taemmini/anaconda3/envs/Depression Analysis/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/taemmini/anaconda3/envs/Depression Analysis/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/taemmini/anaconda3/envs/Depression Analysis/lib/python3.11/site-packages/sklearn/utils/valid

In [68]:
# 칼럼별 데이터타입 확인
expanded_df.dtypes

REF_DATE                   int64
GEO                       object
Sex                        int64
Age group                 object
Frequency                float64
TERMINATED                object
Ailments_Anxiety            bool
Ailments_Backache           bool
Ailments_Depressed          bool
Ailments_Dizziness          bool
Ailments_Headache           bool
Ailments_Insomnia           bool
Ailments_Irritability       bool
Ailments_Stomachache        bool
Ailments_Tiredness          bool
dtype: object

In [69]:
# Terminated Boolean 변환
expanded_df['TERMINATED'] = expanded_df['TERMINATED'].apply(lambda x: True if x == 'True' else False)

# Age group int 변환
expanded_df['Age group'] = expanded_df['Age group'].astype(int)

In [70]:
# 칼럼별 데이터타입 확인
expanded_df.dtypes

REF_DATE                   int64
GEO                       object
Sex                        int64
Age group                  int64
Frequency                float64
TERMINATED                  bool
Ailments_Anxiety            bool
Ailments_Backache           bool
Ailments_Depressed          bool
Ailments_Dizziness          bool
Ailments_Headache           bool
Ailments_Insomnia           bool
Ailments_Irritability       bool
Ailments_Stomachache        bool
Ailments_Tiredness          bool
dtype: object

In [72]:
expanded_df.to_csv('Ailments/Dataset_Aliments/13100196_preprocessed.csv', index=False)